In [23]:
# import required libraries
import pandas as pd
import numpy as np 

In [24]:
# read files 
Game_origin = pd.read_csv('../game with ratings/Games.csv')
Game_sales_origin = pd.read_csv('../game sales with image from vg/game_sales_with_image.csv')
Game_reviews_origin = pd.read_csv('../game reviews from vandal/game_reviews.csv')

In [25]:
# modify Game_origin to get the Game tables 
Game = Game_origin[["id", "name", "released", "developers"]].dropna()
Game["developers"] = Game["developers"].str.replace("||", ", ", regex=False)
Game = Game.rename(columns={'released': 'release_date', 'developers': 'developer'})

In [26]:
# extract Game table to Game.csv
Game.to_csv("Game.csv", index=False)

In [27]:
# modify Game_origin and Game to get Genre and Game_genre tables
Game_genre = Game_origin[["id", "genres"]]
Game_genre = pd.merge(Game, Game_genre, how='left')[["id", "genres"]]
Game_genre["genres"] = Game_genre["genres"].str.split("\|\|")
Game_genre = Game_genre.explode("genres").reset_index()[["id", "genres"]]

Genre = Game_genre[["genres"]]
Genre = Genre.sort_values(by=['genres'])
Genre = Genre[["genres"]].drop_duplicates().dropna().reset_index()[["genres"]]
Genre = Genre.rename(columns={'genres': 'name'})

In [28]:
# extract Game_genre and Genre to Game_genre.csv and Genre.csv respectively
Game_genre.to_csv("Game_genre.csv", index=False)
Genre.to_csv("Genre.csv", index=False)

In [29]:
# clean Game_reviews_origin 
Game_reviews_origin['website_rating'] = Game_reviews_origin['website_rating'].str.replace('None', '')
Game_reviews_origin['website_rating'] = Game_reviews_origin['website_rating'].str.replace(',', '')
Game_reviews_origin['website_rating'] = Game_reviews_origin['website_rating'].str.replace('-', '')
Game_reviews_origin['user_rating'] = Game_reviews_origin['website_rating'].str.replace('None', '')
Game_reviews_origin['user_rating'] = Game_reviews_origin['website_rating'].str.replace(',', '')
Game_reviews_origin['user_rating'] = Game_reviews_origin['website_rating'].str.replace('-', '')
Game_reviews_origin['website_rating'] = pd.to_numeric(Game_reviews_origin['website_rating'])
Game_reviews_origin['user_rating'] = pd.to_numeric(Game_reviews_origin['user_rating'])

In [30]:
# merge Game_reviews_origin and Game to find the user and website rating of the games 
Rating = pd.merge(Game, Game_reviews_origin, left_on='name', right_on='game', how='inner')[['id_x', 'website_rating', 'user_rating']]
Rating = Rating.groupby('id_x').mean().reset_index().rename(columns={'id_x': 'id'})

In [31]:
# extract Rating to Rating.csv
Rating.to_csv("Rating.csv", index=False)

In [32]:
# clean Game_sales_origin
Game_sales_origin["img_url"] = "http://www.vgchartz.com" + Game_sales_origin["img_url"]

In [33]:
# merge Game and Game_sales_origin 
Website = pd.merge(Game, Game_sales_origin, left_on='name', right_on='Name', how='inner')
Website = Website[['id', 'url', 'img_url']].sort_values(by='id').drop_duplicates(subset='id').reset_index(drop=True)
Website = Website.rename(columns={'url': 'vg_url', 'img_url': 'photo_url'})


In [34]:
# export Website to Website.csv
Website.to_csv("Website.csv", index=False)